In [35]:
using DelimitedFiles
using Plots
using Printf

In [36]:
function f(x)
    angle_in_radians = (π/1) * x
    return sin(angle_in_radians)*sin(angle_in_radians)
end
;

In [37]:
function we(nint)
    w = zeros(nint)
    if(nint == 2)
        w[1] = 1.0
        w[2] = 1.0
    elseif(nint == 3)
        w[1] = 5/9
        w[2] = 8/9
        w[3] = 5/9
    elseif(nint == 4)
        w[1] = (18 - sqrt(30.0))/36.0
        w[2] = (18 + sqrt(30.0))/36.0
        w[3] = (18 + sqrt(30.0))/36.0
        w[4] = (18 - sqrt(30.0))/36.0
    elseif(nint == 5)
        w[1] = (322.0-13.0*sqrt(70.0))/900.0
        w[2] = (322.0+13.0*sqrt(70.0))/900.0
        w[3] = 128.0/225.0
        w[4] = (322.0+13.0*sqrt(70.0))/900.0
        w[5] = (322.0-13.0*sqrt(70.0))/900.0
    end
    return w
end
;

In [38]:
function shl(nen,nint)
    pt = zeros(nint)
    w = zeros(nint)
    # print("line 3.2.1")
    if(nint == 2)
        pt[1] = -sqrt(3.)/3.;
        pt[2] = sqrt(3.)/3.;
        w[1] = 1.;
        w[2] = 1.;
    elseif(nint == 3)
        pt[1] = sqrt(3/5);
        pt[2] = 0;
        pt[3] = -sqrt(3/5);
        w[1] = 5/9;
        w[2] = 8/9;
        w[3] = 5/9;
    elseif(nint == 4)
        pt[1] = sqrt((3+2*sqrt(6/5))/7);
        pt[2] = sqrt((3-2*sqrt(6/5))/7);
        pt[3] = -sqrt((3-2*sqrt(6/5))/7);
        pt[4] = -sqrt((3+2*sqrt(6/5))/7);
        w[1] = (18-sqrt(30.))/36;
        w[2] = (18+sqrt(30.))/36;
        w[3] = (18+sqrt(30.))/36;
        w[4] = (18-sqrt(30.))/36;
    elseif(nint == 5)
        pt[1] =  (1/3)*sqrt((5+2*sqrt(10/7)));
        pt[2] =  (1/3)*sqrt((5-2*sqrt(10/7)));
        pt[3] =   0;
        pt[4] = -(1/3)*sqrt((5-2*sqrt(10/7)));
        pt[5] = -(-1/3)*sqrt((5+2*sqrt(10/7)));
        w[1] = (322-13*sqrt(70))/900;
        w[2] = (322+13*sqrt(70))/900;
        w[3] =  128/225;
        w[4] = (322+13*sqrt(70))/900;
        w[5] = (322-13*sqrt(70))/900;
    end
    sh = zeros(nen, nint)
    for l=1:nint
        t=pt[l];
        if(nen==2)
            sh[1,l] = (1.0-t)/2.0;
            sh[2,l] = (1.0+t)/2.0;
        elseif(nen==3)
            sh[1,l] = t*(t-1.0)/2.0;
            sh[2,l] = -(t-1)*(t+1);
            sh[3,l] = t*(t+1.0)/2.0;
        elseif(nen==4)
            sh[1,l] = -( 9/16)*(t+(1/3))*(t-(1/3))*(t-1);
            sh[2,l] =  (27/16)*(t+1)    *(t-(1/3))*(t-1);
            sh[3,l] = -(27/16)*(t+1)    *(t+(1/3))*(t-1);
            sh[4,l] =  ( 9/16)*(t+1)    *(t+(1/3))*(t-(1/3));
        elseif(nen==5)
            sh[1,l] =  (2/3)*(t+(1/2))*        t*(t-(1/2))*(t-1);
            sh[2,l] = -(8/3)*    (t+1)*        t*(t-(1/2))*(t-1);
            sh[3,l] =      4*    (t+1)*(t+(1/2))*(t-(1/2))*(t-1);
            sh[4,l] = -(8/3)*    (t+1)*(t+(1/2))*        t*(t-1);
            sh[5,l] =  (2/3)*    (t+1)*(t+(1/2))*        t*(t-(1/2));	 
        end
    end
    return sh
end
;

In [39]:
folder_path = "graficos/"

if !isdir(folder_path)
    mkdir(folder_path)
end
;

In [40]:
a = -2.0
b = +2.0
;

In [41]:
k = 1
;

In [42]:
function simulate_l2(max_degree, folder_to_save::String)
    error = []
    nell  = []
    xl = []
    n_values = [4^i for i in 1:(max_degree)]
    for nel in n_values
        # fig, axs = plot(layout=(1, 4), size=(800, 200))
        for k in 1:4
            
            np = k * nel + 1 # número de nós da malha
            nen = k + 1 # número de nós no elemento
            nint = k + 1 # número de pontos de integração

            h = (b - a)/nel # espaçamento entre os elementos

            xl = zeros(np)
            xl[1] = a
            for i in 2:np
                xl[i] = xl[i-1] + (h/k)
            end

            # definindo as matrizes globais
            M = zeros(np, np)
            F = zeros(np)
            # montando o problema global
            max_iter = (nel * k - (k - 1))
            for n in 1:max_iter
                Me = zeros(nen, nen)
                Fe = zeros(nen)
                shg = shl(nen, nint)
                w = we(nint)
                for l in 1:nint
                    xx = 0.0
                    for i in 1:nen
                        xx=xx+shg[i,l]*xl[n+i-1]
                    end
                    for j in 1:nen
                        Fe[j] = Fe[j] + f(xx)*shg[j,l]*w[l]*h/2.0
                        for i in 1:nen
                            Me[i,j] = Me[i,j] + shg[i,l]*shg[j,l]*w[l]*h/2.0
                        end
                    end
                end
                for j in 1:nen
                    F[n+j-1] = F[n+j-1] + Fe[j]
                    for i in 1:nen
                        M[n+i-1,n+j-1] = M[n+i-1,n+j-1] + Me[i,j]
                    end
                end
            end
            alpha = zeros(np)
            alpha = M\F
            solucao = alpha
            
            fig = plot(size=(1000, 500))
            plot!(xl, solucao, label="aproximacao L2")  # Continuous line
            plot!(range(a, stop=b, length=100), x -> f(x), seriestype=:scatter, label="analitica", ms=2.5)  # Scatter plot
            title!("Grau = $k - nel = $nel")
            plot!(grid=true)  # Enable the grid
            # savefig("grafico_nel$(nel)_grau$(k).png")
            savefig(joinpath(folder_to_save, "grafico_nel_$(nel)_grau$(k).png"))
            # download("grafico_nel$nel_grau$k.png")  # Uncomment if you want to download the file
            # legend!("upper right")
            # display(fig)
        end
    end

end


simulate_l2 (generic function with 1 method)

In [43]:
function simulate_mef(max_degree, folder_to_save::String)
end
;

In [46]:
subfolder_name = "simulacao_da_projecao_l2"

full_path = joinpath(folder_path, subfolder_name)

if !isdir(full_path)
    mkdir(full_path)
end

elapsed_time = @elapsed begin
    simulate_l2(6, full_path)
end

print("Tempo gasto: ")
@printf("%.4e\n", elapsed_time)



Tempo gasto: 3.6273e-01


In [47]:
subfolder_name = "simulacao_da_mef_questao_1"

full_path = joinpath(folder_path, subfolder_name)

if !isdir(full_path)
    mkdir(full_path)
end

elapsed_time = @elapsed begin
    simulate_mef(6, full_path)
end

print("Tempo gasto: ")
@printf("%.4e\n", elapsed_time)



Tempo gasto: 4.0000e-07
